# Optimal Power Flow IO response analysis and Black Box modelling

In this notebook, we analyze the response of the OPF, aiming at creating a BBM of it using the scenarios generated using RESTART.


In [1]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import dill as pickle
from pathlib import Path

In [2]:
# Show current working directory
print(f"Current working directory: {os.getcwd()}")

Current working directory: D:\projects\IPTLC_BBMs


## First, let's do an example of a single simulation of the OPF response

In [3]:
# Data folder path
data_folder = Path("../CPS_GBM_IPTLC/data/ScenarioGeneration/PowerGrid/MonteCarlo/2023-12-06_18-00-46")

# Target simulation folder
target_folder = data_folder / "0"

# Specify the path to save processed data and create the folder if it doesn't exist
processed_data_folder = Path("data/OPF/IO_response")
os.makedirs(processed_data_folder, exist_ok=True)

In [4]:
# Open the dataframes
sim_u_df = pd.read_csv(target_folder / "external_stimuli.csv", index_col=0)
sim_x_df = pd.read_csv(target_folder / "state.csv", index_col=0)
sim_theta_df = pd.read_csv(target_folder / "condition.csv", index_col=0)

In [5]:
sim_u_df.head()

,Pd,Pd.1,Pd.2,Pd.3,Pd.4,Pd.5,Pd.6,Pd.7,Pd.8,Pd.9,...,Qd.4,Qd.5,Qd.6,Qd.7,Qd.8,Qd.9,Qd.10,Qd.11,Qd.12,Qd.13
0,0.0,14.556323,47.123383,22.191755,4.712971,7.289077,0.0,0.0,13.082205,5.084837,...,1.048428,5.155498,0.0,0.0,7.717014,3.033411,0.721996,0.950130,2.958551,3.101872
900,0.0,13.968725,50.224851,27.714751,4.644591,4.736703,0.0,0.0,12.766659,5.637937,...,0.763172,4.426070,0.0,0.0,9.722176,3.268228,1.054579,0.937279,2.911330,3.048261
1800,0.0,13.032938,48.497201,33.992821,3.780178,6.144648,0.0,0.0,18.406073,6.028764,...,0.826703,3.176092,0.0,0.0,9.095054,2.930612,1.026258,0.813046,3.528570,2.154418
2700,0.0,12.927204,45.545196,27.585807,3.925114,5.252617,0.0,0.0,14.012997,4.497281,...,0.724904,3.481796,0.0,0.0,10.261761,3.396639,0.865064,1.165194,2.178819,3.004248
3600,0.0,10.890928,39.297153,29.272179,4.345949,6.919566,0.0,0.0,14.927784,3.922806,...,0.851476,3.926387,0.0,0.0,9.306241,3.066271,0.967078,1.034497,3.136463,2.433796


In [6]:
sim_x_df.head()

,Pd,Pd.1,Pd.2,Pd.3,Pd.4,Pd.5,Pd.6,Pd.7,Pd.8,Pd.9,...,piLoad.4,piLoad.5,piLoad.6,piLoad.7,piLoad.8,piLoad.9,piLoad.10,piLoad.11,piLoad.12,piLoad.13
0,0.0,14.556323,47.123383,22.191755,4.712971,7.289077,0.0,0.0,13.082205,5.084837,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
900,0.0,13.968725,50.224851,27.714751,4.644591,4.736703,0.0,0.0,12.766659,5.637937,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1800,0.0,13.032938,48.497201,33.992821,3.780178,6.144648,0.0,0.0,18.406073,6.028764,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2700,0.0,12.927204,45.545196,27.585807,3.925114,5.252617,0.0,0.0,14.012997,4.497281,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3600,0.0,10.890928,39.297153,29.272179,4.345949,6.919566,0.0,0.0,14.927784,3.922806,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
sim_theta_df.head()

,R,R.1,R.2,R.3,R.4,R.5,R.6,R.7,R.8,R.9,...,tEventLine.10,tEventLine.11,tEventLine.12,tEventLine.13,tEventLine.14,tEventTrafo,tEventTrafo.1,tEventTrafo.2,tEventTrafo.3,tEventTrafo.4
0,3.532005,9.846967,8.563928,10.590547,10.379138,12.212573,2.433038,0.000041,0.000053,0.000029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
900,3.532005,9.846967,8.563928,10.590547,10.379138,12.212573,2.433038,0.000041,0.000053,0.000029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1800,3.532005,9.846967,8.563928,10.590547,10.379138,12.212573,2.433038,0.000041,0.000053,0.000029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2700,3.532005,9.846967,8.563928,10.590547,10.379138,12.212573,2.433038,0.000041,0.000053,0.000029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3600,3.532005,9.846967,8.563928,10.590547,10.379138,12.212573,2.433038,0.000041,0.000053,0.000029,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Input

The input is the concatenation of the external stimuli and the discrete part of the condition of the system

In [8]:
# Obtain the discrete part of the condition. Those are all the columns that have "pi" in their name
theta_pi_df = sim_theta_df.loc[:, sim_theta_df.columns.str.contains("pi")]
theta_pi_df.head()

,piGen,piGen.1,piGen.2,piGen.3,piGen.4,piLine,piLine.1,piLine.2,piLine.3,piLine.4,...,piLoad.4,piLoad.5,piLoad.6,piLoad.7,piLoad.8,piLoad.9,piLoad.10,piLoad.11,piLoad.12,piLoad.13
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
900,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1800,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2700,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3600,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
# Concatenate the external stimuli and the discrete part of the condition
x = np.concatenate((sim_u_df.values, theta_pi_df.values), axis=1)
x.shape

(97, 81)

In [10]:
# Save the input
input_path = processed_data_folder / "input_single_example.npy"
np.save(input_path, x)

### Output - Active and reactive power of the generators

In [24]:
# The output is the part of the state that constaints the active and reactive power of the generators, i.e., those columns that have "Pg" or "Qg" in their name
Y_PgQg_df = sim_x_df.loc[:, sim_x_df.columns.str.contains("Pg|Qg")]
Y_PgQg_df.head()

,Pg,Pg.1,Pg.2,Pg.3,Pg.4,Qg,Qg.1,Qg.2,Qg.3,Qg.4
0,119.042074,22.097926,1.556064e-11,9.785433e-13,-1.749435e-11,3.290923,8.085947,12.239712,-5.999253,-5.999934
900,122.292513,22.721510,1.299740e-11,-5.171726e-12,-1.527436e-11,3.555777,11.674836,9.071357,-5.999496,-5.999981
1800,129.017916,23.992108,-5.607376e-11,2.874593e-11,8.895616e-11,4.095875,11.311077,8.697293,-5.999608,-5.999997
2700,117.756178,21.853135,-5.791049e-11,7.905790e-12,6.502552e-11,3.158221,8.487028,9.737738,-5.999608,-5.977210
3600,109.866263,20.343933,2.196367e-12,2.578333e-13,-1.769730e-12,2.525934,7.436979,10.577337,-5.999475,-5.957341


In [25]:
# Save the output
output_path = processed_data_folder / "output_single_example.npy"
np.save(output_path, Y_PgQg_df.values)

### Let's train a BBM using the IO data

We will train a two-layer feedforward neural network with 10 neurons in the hidden layer.

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [36]:
# Load the data
X = np.load(input_path)
Y = np.load(output_path)

In [37]:
# Convert to torch tensors
X = torch.from_numpy(X).float()
Y = torch.from_numpy(Y).float()

In [38]:
# Create a TensorDataset
dataset = TensorDataset(X, Y)

In [39]:
# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [50]:
# Define the model: a two-layer feedforward neural network with 10 neurons in the hidden layer
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # The input is of size 81 and the output is of size 10
        self.fc1 = nn.Linear(81, 15)
        self.fc2 = nn.Linear(15, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [51]:
# Initialize the model
net = Net()

In [52]:
# Define the loss function
criterion = nn.MSELoss()

In [53]:
# Define the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [65]:
# Train the model
for epoch in range(1000):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # Get the inputs
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100}")
            running_loss = 0.0

print("Finished training")

Finished training


In [66]:
# Evaluate the model
outputs = net(X)
loss = criterion(outputs, Y)
print(f"Loss: {loss.item()}")

Loss: 0.5179461240768433
